In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import scipy.signal as sig

import reconstruction as recs

%matplotlib inline

In [ ]:
data_dir = os.path.expanduser('~/Google Drive/Bas Zahy Gianni - Games/Data/4_rcn/Clean/Game')
files = [f for f in os.listdir(data_dir) if f[-3:] == 'csv']
filepaths = [os.path.join(data_dir, f) for f in files]
df = pd.concat([pd.read_csv(f) for f in filepaths]).reset_index(drop=True)

In [ ]:
trained = df['trained']
reconf = df['status'] == 'reconf'
reconi = df['status'] == 'reconi'
terminals = reconf | reconi

In [ ]:
df.loc[trained, 'subject'] = '1' + df.loc[trained, 'subject']
df.loc[~trained, 'subject'] = '0' + df.loc[~trained, 'subject']

ipos = df.loc[reconi, ['subject', 'bp', 'wp']].copy().reset_index()
ipos.columns = ['indexi', 'subject', 'bp true', 'wp true']
fpos = df.loc[reconf, ['bp', 'wp']].copy().reset_index()
fpos.columns = ['indexf', 'bp', 'wp']

comp = pd.concat([ipos, fpos], axis=1)

In [ ]:
true_bp = np.stack(df.loc[reconi, 'bp'].map(recs.string_to_array).values)
true_wp = np.stack(df.loc[reconi, 'wp'].map(recs.string_to_array).values)

num_h_neighbors = np.apply_along_axis(recs.h_neighbors, axis=1, arr=true_bp + true_wp)
num_v_neighbors = np.apply_along_axis(recs.v_neighbors, axis=1, arr=true_bp + true_wp)
num_d_neighbors = np.apply_along_axis(recs.d_neighbors, axis=1, arr= true_bp + true_wp)
total_neighbors = num_h_neighbors + num_v_neighbors + num_d_neighbors

In [ ]:
total_neighbors.reshape([num_h_neighbors.shape[0], 4, 9])[0, :, :]

In [ ]:
def count_h_neighbors(row):
    bpt, wpt, bp, wp = recs.expand_row(row)
    f = np.array([[1, 0, 1]])
    
    bpt_ = bpt.reshape([4, 9])
    bpt_ = sig.convolve2d(bpt_, f, mode='same').reshape(36)
    
    wpt_ = wpt.reshape([4, 9])
    wpt_ = sig.convolve2d(wpt_, f, mode='same').reshape(36)
    
    return bpt_ + wpt_

comp.apply(count_h_neighbors, axis=1)

In [ ]:
npieces = comp.apply(recs.n_pieces, axis=1)
missing = comp.apply(recs.missing_pieces, axis=1)
extra = comp.apply(recs.extra_pieces, axis=1)
wrong = comp.apply(recs.wrong_color, axis=1)

In [ ]:
df.loc[reconf, 'total errors'] = missing.values + extra.values + wrong.values
df.loc[reconf, 'n initial'] = npieces.values
df.loc[reconf, 'n missing'] = missing.values
df.loc[reconf, 'n extra'] = extra.values
df.loc[reconf, 'n wrong_color'] = wrong.values

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(8, 5))
trained_errors = df.loc[trained & reconf, 'total errors'].values
untrained_errors = df.loc[(~trained) & reconf, 'total errors'].values

label = 'Trained: {:.2f}'.format(trained_errors.mean())
sns.distplot(trained_errors, ax=axes, label=label, kde=False, bins=np.arange(0, 36, 1))

label = 'Untrained: {:.2f}'.format(untrained_errors.mean())
sns.distplot(untrained_errors, ax=axes, label=label, kde=False, bins=np.arange(0, 36, 1))

axes.legend()
sns.despine()

In [ ]:
# Ref here for method: https://onlinecourses.science.psu.edu/stat414/node/268

n0 = trained_errors.size * 36              # Num possible error events = num trials * num positions
n1 = untrained_errors.size * 36
p0 = trained_errors.sum() / n0
p1 = untrained_errors.sum() / n1

phat = ((n0 * p0) + (n1 * p1)) / (n0 + n1)

z = (p0 - p1) / np.sqrt(phat * (1 - phat) * (1/n0 + 1/n1))
print('Trials per group: {}\nZ-Stat: {:.2f}'.format(n0/36, np.abs(z)))